<a href="https://colab.research.google.com/github/AdamAnderB/Bramlett_DSPN_S24/blob/main/book/exercises/power-analysis-via-simulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 15: Power analyses

This  assignment is designed to give you practice with Monte Carlo methods to conduct power analyses via simulation. You won't need to load in any data for this homework. We will, however, be using parts of the homework from last week.

---
## 1. Simulating data (1 points)


Pull your `simulate_data()` function from your last homework and add it below.

As a reminder, this function simulates the relationship between age, word reading experience, and reading comprehension skill.

`c` is reading comprehension, and `x` is word reading experience.

In [2]:
sample_size = 100 # How many children in data set?
age_lo = 80     # minimum age, in months
age_hi = 200    # maximum age, in months
beta_xa = 0.5   # amount by which experience changes for increase of one month in age
beta_x0 = -5    # amount of experience when age = 0 (not interpretable, since minimum age for this data is 80 months)
sd_x = 50       # standard dev of gaussian noise term, epsilon_x
beta_ca = 0.8   # amount that comprehension score improves for every increase of one unit in age
beta_cx = 3     # amount that comprehension score improves for every increase of one unit in reading experience
beta_c0 = 10    # comprehension score when reading experience is 0.
sd_c = 85      # standard dev of gaussian noise term, epsilon_c

simulate_data <- function(sample_size, age_lo, age_hi, beta_xa,beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
        age <- runif(sample_size, age_lo, age_hi)
        x <- beta_x0 + beta_xa * age + rnorm(sample_size, mean = 0, sd = sd_x)
        c <- beta_c0 + beta_ca * age + beta_cx * x + rnorm(sample_size, mean = 0, sd = sd_c)
  data <- data.frame(age = age, reading_ex = x, comprehension = c)

}

dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
head(dat)
dim(dat)

,age,reading_ex,comprehension
,<dbl>,<dbl>,<dbl>
1,167.1041,18.00757,179.4387
2,139.0855,78.71473,351.3369
3,100.9920,50.21623,394.0022
4,164.5187,42.27462,303.7619
5,110.9981,108.22190,604.3042
6,173.0078,144.27482,576.5362


[1] 100   3

---
## 2. `run_analysis()` function (2 pts)

Last week, we looked at whether word reading experience(`x`) mediated the relation between `age` and reading comprehension (`c`).

Now we're going to use our `simulate_data()` function to conduct a power analysis. The goal is to determine how many participants we would need in order to detect both the mediated and the direct effects in this data.

*Note: We're going to pretend for the sake of simplicity that we don't have any control over the ages of the children we get (so ages are generated using `runif(sample_size, age_lo, age_hi)`, although of course this would be an unusual situation in reality.*

First, write a function, `run_analysis()`, that takes in simulated data, runs **your mediation from last week**, and returns a vector containing the ACME and ADE estimates and p-values (these are the `d0`, `d0.p`, `z0`, and `z0.p` features of the mediated model object, e.g., `fitMed$d0.p`). Print this function's output for the data we simulated previously.

In [5]:
# WRITE YOUR CODE HERE
#install.packages("mediation")
library(mediation)
run_analysis <- function(data) {
  mediator_model <- lm(reading_ex ~ age, data = data)
  outcome_model <- lm(comprehension ~ age + reading_ex, data = data)
  med_fit <- mediate(mediator_model, outcome_model, treat = "age", mediator = "reading_ex")
  results <- c(ACME = summary(med_fit)$d0, ACME_p = summary(med_fit)$d0.p,
               ADE = summary(med_fit)$z0, ADE_p = summary(med_fit)$z0.p)
  return(results)
}

analysis_results <- run_analysis(dat)
analysis_results

ACME    ACME_p       ADE     ADE_p 
1.8024402 0.0000000 0.2029004 0.4580000

---
## 3. `repeat_analysis()` function (3 pts)

Next fill in the function `repeat_analysis()` below so that it simulates and analyzes data `num_simulations` times. Store the outputs from each simulation in the `simouts` matrix. Calculate and return the coverage across all the simulations run for both ACME and ADE.

In [24]:
repeat_analysis <- function(num_simulations, alpha, sample_size, age_lo, age_hi,
        beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c) {
    # Initialize simouts matrix for storing each output from run_analysis()
    simouts <- matrix(rep(NA, num_simulations * 4), nrow = num_simulations, ncol = 4,
                      dimnames = list(NULL, c("ACME", "ACME_p", "ADE", "ADE_p")))

    # Start simulating
    for (i in 1:num_simulations) {
      dat <- simulate_data(sample_size, age_lo, age_hi, beta_xa, beta_x0, sd_x, beta_ca, beta_cx, beta_c0, sd_c)
      results <- run_analysis(dat)
      simouts[i, ] <- unlist(results)
    }

    # Calculate coverage for both ACME and ADE estimates using p-values in simouts
    ACME_cov = mean(simouts[, "ACME_p"] <= alpha)# FILL THIS IN
    ADE_cov =  mean(simouts[, "ADE_p"] <= alpha)# FILL THIS IN

    return(list(ACME_cov = ACME_cov, ADE_cov = ADE_cov))
}

Now run the `repeat_analysis()` function using the same parameter settings as above, for 10 simulations, with an alpha criterion of 0.01.

> Add blockquote



In [25]:
# WRITE YOUR CODE HERE
simulation_results <- repeat_analysis(
    num_simulations = 10,
    alpha = 0.01,
    sample_size = 100,
    age_lo = age_lo,
    age_hi = age_hi,
    beta_xa = beta_xa,
    beta_x0 = beta_x0,
    sd_x = sd_x,
    beta_ca = beta_ca,
    beta_cx = beta_cx,
    beta_c0 = beta_c0,
    sd_c = sd_c
)

simulation_results

$ACME_cov
[1] 0.9

$ADE_cov
[1] 0.8

---
## 4. Testing different sample sizes (2 pts)

Finally, do the same thing (10 simulations, alpha criterion of 0.01) but for 5 different sample sizes: 50, 75, 100, 125, 150. You can do this using `map` (as in the tutorial), or a simple `for` loop, or by calculating each individually. Up to you! This should take around 3 minutes to run.

In [26]:
# WRITE YOUR CODE HERE
simulation_results_50 <- repeat_analysis(
    num_simulations = 50,
    alpha = 0.01,
    sample_size = 100,
    age_lo = age_lo,
    age_hi = age_hi,
    beta_xa = beta_xa,
    beta_x0 = beta_x0,
    sd_x = sd_x,
    beta_ca = beta_ca,
    beta_cx = beta_cx,
    beta_c0 = beta_c0,
    sd_c = sd_c
)
simulation_results_50

simulation_results_75 <- repeat_analysis(
    num_simulations = 75,
    alpha = 0.01,
    sample_size = 100,
    age_lo = age_lo,
    age_hi = age_hi,
    beta_xa = beta_xa,
    beta_x0 = beta_x0,
    sd_x = sd_x,
    beta_ca = beta_ca,
    beta_cx = beta_cx,
    beta_c0 = beta_c0,
    sd_c = sd_c
)
simulation_results_75

simulation_results_100 <- repeat_analysis(
    num_simulations = 100,
    alpha = 0.01,
    sample_size = 100,
    age_lo = age_lo,
    age_hi = age_hi,
    beta_xa = beta_xa,
    beta_x0 = beta_x0,
    sd_x = sd_x,
    beta_ca = beta_ca,
    beta_cx = beta_cx,
    beta_c0 = beta_c0,
    sd_c = sd_c
)
simulation_results_100

simulation_results_125 <- repeat_analysis(
    num_simulations = 125,
    alpha = 0.01,
    sample_size = 100,
    age_lo = age_lo,
    age_hi = age_hi,
    beta_xa = beta_xa,
    beta_x0 = beta_x0,
    sd_x = sd_x,
    beta_ca = beta_ca,
    beta_cx = beta_cx,
    beta_c0 = beta_c0,
    sd_c = sd_c
)
simulation_results_125

simulation_results_150 <- repeat_analysis(
    num_simulations = 150,
    alpha = 0.01,
    sample_size = 100,
    age_lo = age_lo,
    age_hi = age_hi,
    beta_xa = beta_xa,
    beta_x0 = beta_x0,
    sd_x = sd_x,
    beta_ca = beta_ca,
    beta_cx = beta_cx,
    beta_c0 = beta_c0,
    sd_c = sd_c
)
simulation_results_150

$ACME_cov
[1] 0.84

$ADE_cov
[1] 0.66

$ACME_cov
[1] 0.88

$ADE_cov
[1] 0.72

$ACME_cov
[1] 0.82

$ADE_cov
[1] 0.69

$ACME_cov
[1] 0.792

$ADE_cov
[1] 0.712

$ACME_cov
[1] 0.8266667

$ADE_cov
[1] 0.7266667

Print your results.

In [27]:
# WRITE YOUR CODE HERE
simulation_results_50
simulation_results_75
simulation_results_100
simulation_results_125
simulation_results_150

$ACME_cov
[1] 0.84

$ADE_cov
[1] 0.66

$ACME_cov
[1] 0.88

$ADE_cov
[1] 0.72

$ACME_cov
[1] 0.82

$ADE_cov
[1] 0.69

$ACME_cov
[1] 0.792

$ADE_cov
[1] 0.712

$ACME_cov
[1] 0.8266667

$ADE_cov
[1] 0.7266667

## 5. Reflection (2 pts)

If this were a real power analysis, we'd want to run more simulations per sample size (to get a more precise estimate of power) and we may also want to test out some other values of the parameters we used to simulate our data. However, what would you conclude just based on the results above?

> *Write your response here.*
> It seems the the mediated effet is stronger than the direct effect. this could also be due to the way that noise was added in the simulation. It seems that power generally increaeses but also fluxautes with more samples. I would want to do larger samples to draw any heavier conclusions. Perhaps around 500.

**Given** how we generated the data, why was the direct effect harder to detect than the mediated effect?
> *Write your response here.*

**DUE:** 5pm EST, April 3, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*